<h1><font color='blue'>Lending Club</font></h1>

In [3]:
# Import the python environment libraries
import sklearn
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score, KFold, train_test_split
from sklearn import neighbors
from sklearn.metrics import classification_report
from __future__ import print_function
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
import time
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import collections
%matplotlib inline

In [8]:
# Data parsing and importing 

'''
data.csv
'''
data = pd.read_csv("data.csv", parse_dates = [1])

'''
parsing data
'''
df_date_split = pd.DatetimeIndex(data['issue_d'])
data['issue_year'] = df_date_split.year
data['issue_month'] = df_date_split.month


data.head(1)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,issue_year,issue_month
0,55521,107577,1000,1000,0.0,36 months,0.1608,35.2,F,F2,...,42439.0,538.97,42470.0,42439.0,0.0,NaN,1,INDIVIDUAL,2016,7
